## Background 

The world has recently witnessed an outbreak of a respiratory disease caused by a novel (new) coronavirus that was first detected in China and which has now been detected in more than 150 locations internationally. The virus has been named “SARS-CoV-2” and the disease it causes has been named “coronavirus disease 2019” (abbreviated “COVID-19”).

On January 30, 2020, the International Health Regulations Emergency Committee of the World Health Organization (WHO) declared the outbreak a “public health emergency of international concern” (PHEIC). On March 11, WHO publicly characterized COVID-19 as a pandemic.

![CoVid-19 Pandemic](https://article.images.consumerreports.org/f_auto/prod/content/dam/CRO-Images-2020/Health/03Mar/CR-Health-Inlinehero-coronavirus-update-v2-0320)

In this kernel, we will explore data from the evolving CoVid-19 situation to get a better sense of what potentially lies ahead.


## What does science tell us about pandemics?



As coronavirus continues to spread across the world, we see countries increasingly impose strict measures that heavily disrupt daily life. 

These measures are necessary to do what epidemiologists call as **"flattening the curve"**. The idea is to increase social distancing and slow down the virus, so that you don't get a huge spike in the number of people getting sick all at once. 


![FlattentheCurve](https://www.gannett-cdn.com/presto/2020/03/11/USAT/7b03cfcf-ea80-4a94-b518-f401e7e1bb26-flattening_the_curve.jpg)

If we do not take any protective measures, there wouldn't be enough hospital beds or mechanical ventilators for everyone who needs them, and the hospital system would be overwhelmed with cases at once. 

In [ ]:
##  Library Imports
import os
import sys
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from collections import Counter, defaultdict
from plotly import tools
import plotly.offline as py
import plotly.express as px
import plotly.graph_objs as go
import folium
import warnings
warnings.filterwarnings('ignore')
py.init_notebook_mode(connected=True)
import plotly.io as pio
pio.templates.default = "plotly_dark"
# color pallette
cnf = '#FDFBFB' # confirmed - white
dth = '#ff2e63' # death - red
rec = '#21bf73' # recovered - cyan
act = '#fe9801' # active case - yellow

## Data Sources

The primary data for this kernel has been sourced from the repository maintained by Johns Hopkins University Center for Systems Science and Engineering (JHU CSSE). The raw data is refreshed daily and can be accesed from their github repository available in this [link](https://github.com/CSSEGISandData/COVID-19).

For this analysis, we use the following freely available data sources, 
- Daily data for CoVid19 cases split by region
- Geo-Spatial data with Latitudes & Longitudes
- Demography data for countries

In [ ]:
df_master = pd.read_csv("../input/novel-corona-virus-2019-dataset/covid_19_data.csv")
df_master.rename(columns={'Country/Region':'Country'}, inplace=True)
df_master.rename(columns={'Province/State':'State'}, inplace=True)
df_master.rename(columns={'ObservationDate':'Date'}, inplace=True)
# Rename country labels
df_master['Country'].replace(['Mainland China'], 'China',inplace=True)
df_master['Country'].replace(['US'], 'United States',inplace=True)
df_master['Country'].replace(['UK'], 'United Kingdom',inplace=True)

In [ ]:
## Load Lat Long 
coordinates = pd.read_csv('/kaggle/input/latitude-and-longitude-for-every-country-and-state/world_country_and_usa_states_latitude_and_longitude_values.csv')
country_coordinates = coordinates[['country_code','latitude','longitude','country']]
# Load Population
population = pd.read_csv('/kaggle/input/countries-of-the-world/countries of the world.csv',converters={'Country': str.strip})
population = population[['Country','Region','Population']]

Let's take a quick look at the primary CoVid-19 dataset that is imported.

In [ ]:
df_master.head(3)

In [ ]:
# Create derived column called Active Cases 
df_master['Active'] = df_master['Confirmed'] - df_master['Recovered'] - df_master['Deaths']
# Convert date column to appropriate data type
df_master['Date'] = pd.to_datetime(df_master['Date'])

## Data Visualization

We shall attempt to answer some of the key questions around CoVid-19 with the help of visualizations. These visuals shall be refreshed periodically as the outbreak is still an evolving situation.

### What is the latest tally of Covid-19 globally?

In [ ]:
temp = df_master.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop=True)
temp.style.background_gradient(cmap='Set1')

In [ ]:
cur_date = str(temp['Date'][0].date())
print(f"As of {cur_date}, we have {round((temp['Recovered'][0]/temp['Confirmed'][0])*100,3)}% of the infected who have recovered from the virus globally")

In [ ]:
tm = temp.melt(id_vars="Date", value_vars=['Active', 'Deaths', 'Recovered'])
fig = px.treemap(tm, path=["variable"], values="value", height=400, width=600,
                 color_discrete_sequence=[rec, act, dth])
title = f"CoVid19 case split as of {cur_date}"
fig.update_layout(title=title)
fig.show()

In [ ]:
print(f"We can see from the tree map visualization above that {round((temp['Active'][0]/temp['Confirmed'][0])*100,2)}% infected cases worldwide are currently active.")

### What has been the impact of CoVid-19 on different countries?

Let's visualize the latest CoVid-19 numbers from a geo-spatial lens to get a better understanding of the spread in different regions across the globe.

In [ ]:
temp = df_master.copy()
temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop=True)
temp = temp.groupby(['Country'])['Confirmed','Recovered','Deaths','Active'].sum().sort_values(by='Confirmed',ascending=False).reset_index()
# Merge with country coordinates
temp = temp.merge(country_coordinates, left_on="Country", right_on="country")

In [ ]:
population = population[['Country','Region','Population']]
# Merge With Population
temp = temp.merge(population, left_on="Country", right_on="Country")
# Derive Columns 
temp['ConfirmedPer10kPeople'] = ( temp['Confirmed'] / temp['Population'] ) * 10000
temp['DeathsPer10kPeople'] = ( temp['Deaths'] / temp['Population'] ) * 10000

In [ ]:
fig = px.choropleth(temp, 
                    locations="Country", 
                    color="Confirmed",
                    locationmode = 'country names',
                    hover_name="Country",
                    range_color=[0,round(temp['Confirmed'].max())], 
                    color_continuous_scale="peach",
                    title = "CoVid-19 : Confirmed Cases")

fig.show()

In [ ]:
fig = px.choropleth(temp, 
                    locations="Country", 
                    color="Deaths",
                    locationmode = 'country names',
                    hover_name="Country",
                    range_color=[0,round(temp['Deaths'].max())], 
                    color_continuous_scale='portland',
                    title = "CoVid-19 : Death Cases")

fig.show()

In [ ]:
# Filter countries with atleast 5 million people to avoid skewing the scales
is_1mn = temp['Population'] >= 1000000
temp = temp[is_1mn]

The absolute number of confirmed cases reported in each region may not be a useful metric to compare across countries as they have vastly different demographies. To overcome this, we normalize the absolute number of cases with the population size and plot them to get a better intuition of the extent of impact on each country.

In [ ]:
fig = px.choropleth(temp, 
                    locations="Country", 
                    color="ConfirmedPer10kPeople",
                    locationmode = 'country names',
                    hover_name="Country",
                    range_color=[0,round(temp['ConfirmedPer10kPeople'].max())], 
                    color_continuous_scale="peach",
                    title = "Confirmed Cases Per 10000 People")

fig.show()

In [ ]:
fig = px.choropleth(temp, 
                    locations="Country", 
                    color="DeathsPer10kPeople",
                    locationmode = 'country names',
                    hover_name="Country",
                    range_color=[0,round(temp["DeathsPer10kPeople"].max())], 
                    color_continuous_scale='portland',
                    title = "Deaths Per 10000 People")

fig.show()

### How has the trend for CoVid-19 been globally?

In [ ]:
def visualize_trends(fig, region, data, done=True):
    trace1 = go.Scatter(x=data['Date'],
                        y=data['Confirmed'],
                        mode='lines+markers',
                        name='Confirmed',
                        marker=dict(color=cnf,))

    trace2 = go.Scatter(x=data['Date'],
                        y=data['Recovered'],
                        mode='lines+markers',
                        name='Recovered',
                        marker=dict(color=rec,))

    trace3 = go.Scatter(x=data['Date'],
                        y=data['Deaths'],
                        mode='lines+markers',
                        name='Deaths',
                        marker=dict(color=dth,))

    trace4 = go.Scatter(x=data['Date'],
                        y=data['Active'],
                        mode='lines+markers',
                        name='Active',
                        marker=dict(color=act,))

    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.add_trace(trace3)
    fig.add_trace(trace4)
    title = f"{region} Trend for CoVid-19 Cases"
    fig.update_layout(xaxis_title="Date", yaxis_title="Count", title=title)
    if done:
        fig.show()
    

In [ ]:
temp = df_master.groupby('Date')['Confirmed','Recovered','Deaths','Active'].sum().reset_index()
fig = go.Figure()
visualize_trends(fig=fig,
                 region="Global",
                 data=temp)

We notice that the global trend for confirmed cases was exponentially rising  **until the first week of February** primarily driven by cases in **Mainland China**. 

The rate of increase of confirmed cases appeared to stabilize in the **second week of February** before a second wave kicked off on **February 13th**. 

Around **March 13** we notice a **third steep rise** in confirmed cases that can be linked to sustained community transmissions in multiple European countries and the United States. 


### Can we drill down to understand the trend in China vs RoW? 

In [ ]:
is_china = df_master['Country'] == 'China'
temp = df_master[is_china].groupby('Date')['Confirmed','Recovered','Deaths','Active'].sum().reset_index()
fig = go.Figure()
visualize_trends(fig=fig,
                 region="Mainland China",
                 data=temp,
                 done=False)
fig.add_shape(
        dict(
            type="line",
            x0='2020-01-23',
            y0=-10000,
            x1='2020-01-23',
            y1=10000,
            line=dict(
                color="Blue",
                width=3
            )))
fig.add_shape(
        dict(
            type="line",
            x0='2020-02-13',
            y0=50000,
            x1='2020-02-13',
            y1=70000,
            line=dict(
                color="Purple",
                width=3
            )
))
fig.add_shape(
        dict(
            type="line",
            x0='2020-02-20',
            y0=65000,
            x1='2020-02-20',
            y1=85000,
            line=dict(
                color="Green",
                width=3
            )
))
fig.show()

Trends above indicate that China has "flattened the curve" with the recovered cases slowly creeping up and active cases reducing sharply. China witnessed an exponential increase in confirmed cases from the last week of January until the third week of February as the impact of the lockdown took time to be fully realized. 

The colored vertical lines indicate the dates of intervention measures. The blue line indicates when Wuhan was locked down and the purple line shows when the factories in China were shut.

The number of new cases reduced post this period as the draconian lockdown measures implemented seem to have largely helped in containing the CoVid-19 outbreak. It however took nearly a month from when Wuhan was locked down for the impact to be fully realized in terms of curve flattening.

In [ ]:
is_row = df_master['Country'] != 'Mainland China'
temp = df_master[is_row].groupby('Date')['Confirmed','Recovered','Deaths','Active'].sum().reset_index()
fig = go.Figure()
visualize_trends(fig=fig,
                 region="Rest of World",
                 data=temp)

On the other hand, rest of the world is still in the exponential growth phase with daily confirmed numbers still rapidly climbing and newer countries witnessing sustained community transmission.

### How has the Covid-19 outbreak impacted Europe?

In [ ]:
is_italy = df_master['Country'] == 'Italy'
temp = df_master[is_italy].groupby('Date')['Confirmed','Recovered','Deaths','Active'].sum().reset_index()
fig = go.Figure()
visualize_trends(fig=fig,
                 region="Italy",
                 data=temp, 
                 done=False)
fig.add_shape(
        dict(
            type="line",
            x0='2020-03-09',
            y0=5000,
            x1='2020-03-09',
            y1=15000,
            line=dict(
                color="deeppink",
                width=3
            )))
fig.show()

In [ ]:
is_spain = df_master['Country'] == 'Spain'
temp = df_master[is_spain].groupby('Date')['Confirmed','Recovered','Deaths','Active'].sum().reset_index()
fig = go.Figure()
visualize_trends(fig=fig,
                 region="Spain",
                 data=temp, 
                 done=False)
fig.add_shape(
        dict(
            type="line",
            x0='2020-03-15',
            y0=6000,
            x1='2020-03-15',
            y1=10000,
            line=dict(
                color="deeppink",
                width=3
            )))
fig.show()

In [ ]:
is_france = df_master['Country'] == 'France'
temp = df_master[is_france].groupby('Date')['Confirmed','Recovered','Deaths','Active'].sum().reset_index()
fig = go.Figure()
visualize_trends(fig=fig,
                 region="France",
                 data=temp, 
                 done=False)
fig.add_shape(
        dict(
            type="line",
            x0='2020-03-15',
            y0=3000,
            x1='2020-03-15',
            y1=6000,
            line=dict(
                color="deeppink",
                width=3
            )))
fig.show()

In most European countries like Italy, France and Spain sustained community transmission can be observed from the trend seen after the 2nd week of March. 

The pink vertical line indicates when each of the European countries went into lockdown. We should hopefully witness the impact of intervention measures on the trend lines in the coming weeks. 

### What trends do we observe in UK and US?

In [ ]:
is_us = df_master['Country'] == 'United States'
temp = df_master[is_us].groupby('Date')['Confirmed','Recovered','Deaths','Active'].sum().reset_index()
fig = go.Figure()
visualize_trends(fig=fig,
                 region="US",
                 data=temp)

In US, since testing kits were widely accessible only after second week of March, the number of confirmed cases have taken off post **March 18th**. It appears that there has been community transmission in the US much before this point and the lack of test kits resulted in a lot of suspected cases going unreported.


In [ ]:
is_uk = df_master['Country'] == 'United Kingdom'
temp = df_master[is_uk].groupby('Date')['Confirmed','Recovered','Deaths','Active'].sum().reset_index()
fig = go.Figure()
visualize_trends(fig=fig,
                 region="UK",
                 data=temp, 
                 done=False)

fig.add_shape(
        dict(
            type="line",
            x0='2020-03-22',
            y0=5000,
            x1='2020-03-22',
            y1=6500,
            line=dict(
                color="deeppink",
                width=3
            )))
fig.show()

On the other hand, UK imposed lockdown measures on the 22nd of March when the confirmed case count was already above 5000 and there was widespread community transmission. It appears that UK still has a few weeks of exponential growth ahead before the impact of lockdown can be observed in the trends.

### How has CoVid-19 trended in countries with proactive government interventions (Singapore and South Korea)?

In [ ]:
is_singapore = df_master['Country'] == 'Singapore'
temp = df_master[is_singapore].groupby('Date')['Confirmed','Recovered','Deaths','Active'].sum().reset_index()
fig = go.Figure()
visualize_trends(fig=fig,
                 region="Singapore",
                 data=temp)

Singapore was one of the first few countries to witness imported cases from China in the last week of January. Despite this, the island nation has not witnessed an exponential growth  due to their meticulous contact tracing and early isolation of CoVid-19 cases. 

They have shown other countries that proactive government intervention and meticulous contact tracing early on is really pivotal in slowing down the spread of the virus.


In [ ]:
is_sk = df_master['Country'] == 'South Korea'
temp = df_master[is_sk].groupby('Date')['Confirmed','Recovered','Deaths','Active'].sum().reset_index()
fig = go.Figure()
visualize_trends(fig=fig,
                 region="South Korea",
                 data=temp, 
                 done=False)
fig.add_shape(
        dict(
            type="line",
            x0='2020-02-29',
            y0=2000,
            x1='2020-02-29',
            y1=4000,
            line=dict(
                color="Blue",
                width=3
            )))
fig.show()


South Korea could not avoid a local community transmission as seen from the trend in the second half of February. 

However, through mass testing introduced on February 29th shown using the vertical blue line, they seem to have slowed the exponential increase in March and flattened their curve early. This has prevented their healthcare systems from being overwhelmed thereby ensuring a low mortality rate. 

The experiences of South Korea shows that rapid large scale testing is another key intervention measure that can help with early flattening of the curve. 

### How has the CoVid-19 trend been for India?

In [ ]:
is_india = df_master['Country'] == 'India'
temp = df_master[is_india].groupby('Date')['Confirmed','Recovered','Deaths','Active'].sum().reset_index()
fig = go.Figure()
visualize_trends(fig=fig,
                 region="India",
                 data=temp, 
                 done=False)

fig.add_shape(
        dict(
            type="line",
            x0='2020-03-22',
            y0=300,
            x1='2020-03-22',
            y1=500,
            line=dict(
                color="deeppink",
                width=3
            )))
fig.show()

### How did CoVid-19 spread from China to RoW over time?

In [ ]:
## Extract top countries with most cases
temp = df_master.copy()
temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop=True)
top10_df = temp.groupby(['Country'])['Confirmed','Recovered','Deaths','Active'].sum().sort_values(by='Confirmed',ascending=False).head(8)
top10_countries = top10_df.index.values

## Filter for the top 10 countries in base data and plot for confirmed per 10k people and deaths per 10k people
temp = df_master.copy()
## Merge With Population
temp = temp.merge(population, left_on="Country", right_on="Country")
# Derive Columns 
temp['ConfirmedPer10kPeople'] = ( temp['Confirmed'] / temp['Population'] ) * 10000
temp['DeathsPer10kPeople'] = ( temp['Deaths'] / temp['Population'] ) * 10000

# Aggregate at country, date level and filter data for top 10 countries only
temp = temp.groupby(['Country','Date'])['ConfirmedPer10kPeople','DeathsPer10kPeople'].mean().reset_index()
is_top10 = temp['Country'].isin(top10_countries)
temp = temp[is_top10]

# Filter out cases from Feb 25 when multiple countries started showing cases
temp = temp[temp['Date'] >= datetime.date(2020,2,25)]

# Convert date to correct format 
temp["Date"] = pd.to_datetime(temp["Date"] , format="%m/%d/%Y").dt.date
temp.sort_values(by=["Country","Date"]).reset_index(drop=True)
temp["Date"] = temp["Date"].astype(str)

In [ ]:
fig = go.Figure()
fig = px.bar(temp,
                x="Country",
                y="ConfirmedPer10kPeople",
                color="Country",
                animation_frame="Date",
                animation_group="Country",
                title="Time Lapse - Confirmed Cases Per 10000 People")
fig.show()

In [ ]:
fig = go.Figure()
fig = px.bar(temp,
                x="Country",
                y="DeathsPer10kPeople",
                color="Country",
                animation_frame="Date",
                animation_group="Country",
                title="Time Lapse - Deaths Per 10000 People")
fig.show()

### What stage of this epidemic are we in currently?

A handy measure to evaluate the progression of an epidemic is the **Growth Rate**. 

Growth rate can be defined as the number of new confirmed cases on *day t* divided by the number of new confirmed cases on *day t-1*. 

During the sustained community transmission phase, growth rate is larger than 1 consistently indicating that the number of new cases is increasing at an exponential fashion every day. At some point of the epidemic we reach an inflection point, after which the growth rate drops to less than 1. The inflection point refers to the peak stage of the epidemic.

Let us try to compute and visualize the growth rate for the CoVid-19 outbreak in different parts of the globe where there has been community transmission of the virus. This would give a better idea about which stage of the epidemic are the different countries going through.

In [ ]:
temp = df_master[['State','Country','Date','Confirmed']].copy()


### Growth rate visualization in China (within and outside the Hubei province)

In [ ]:
is_hubei = (temp['State'].isin(['Hubei'])) & (temp['Country'].isin(['China']))
temp_hubei = temp[is_hubei]
is_ro_china = (temp['Country'].isin(['China'])) & (temp['State'] != 'Hubei')
temp_ro_china = temp[is_ro_china]
temp_ro_china = temp_ro_china.groupby(['Country','Date'])['Confirmed'].sum().reset_index()

def growth_rate_compute(temp):
    """
    Function to derive the growth rate column for given region
    """
    temp = temp.sort_values(by='Date')
    temp['Confirmed_t-1'] = temp['Confirmed'].shift(1, axis=0)
    temp = temp[temp['Confirmed_t-1'].notna()]
    temp.reset_index(inplace=True, drop=True)
    temp['New_Cases_t'] = temp['Confirmed'] - temp['Confirmed_t-1']
    # Filter the data points on days with New_Cases_t is zero (reporting anomalies)
    temp = temp[temp['New_Cases_t'] != 0.0 ]
    temp['New_Cases_t-1'] = temp['New_Cases_t'].shift(1, axis=0)
    temp['Growth_Rate'] = temp['New_Cases_t'] / temp['New_Cases_t-1']
    # Remove anomalies in growth rate (excess of > 10) 
    temp = temp[temp['Growth_Rate'] <= 20.0 ]
    temp = temp[temp['Growth_Rate'].notna()]
    return temp

temp_hubei = growth_rate_compute(temp=temp_hubei)
temp_ro_china = growth_rate_compute(temp=temp_ro_china)

In [ ]:
fig = go.Figure()
#trace1 = go.Bar(x=temp_hubei['Date'], 
 #               y=temp_hubei['Growth_Rate'],
 #               hovertext = "Hubei,China",
 #               marker=dict(color="red",))

trace1 = go.Scatter(x=temp_hubei['Date'],
                    y=temp_hubei['Growth_Rate'],
                    mode='lines+markers',
                    hovertext='Hubei,China',
                    marker=dict(color="red",))

fig.add_trace(trace1)
fig.update_layout(title=' Growth Rate Trend in Hubei, China')
fig.show()

In [ ]:
fig = go.Figure()
trace1 = go.Scatter(x=temp_ro_china['Date'],
                    y=temp_ro_china['Growth_Rate'],
                    mode='lines+markers',
                    hovertext='Rest of China',
                    marker=dict(color="orange",))

fig.add_trace(trace1)
fig.update_layout(title=' Growth Rate Trend in China (Outside Hubei)')
fig.show() 

In Hubei (China) which is the epicenter of the outbreak, growth rates over the past 2 weeks have been consistently less than 1 indicating that they are past the inflection point in Hubei. 

In rest of china, although the growth rate seem to be fluctuating, the actual figure of new cases daily is substantially low (in low double digits). 

### Growth rate visualization in Europe (Spain, Italy)

In [ ]:
is_italy = temp['Country'] == 'Italy'
temp_italy = temp[is_italy]
temp_italy = temp_italy.groupby(['Country','Date'])['Confirmed'].sum().reset_index()
temp_italy = growth_rate_compute(temp=temp_italy)
fig = go.Figure()
trace1 = go.Scatter(x=temp_italy['Date'],
                    y=temp_italy['Growth_Rate'],
                    mode='lines+markers',
                    hovertext='Italy',
                    marker=dict(color="red",))

fig.add_trace(trace1)
fig.update_layout(title=' Growth Rate Trend in Italy')
fig.show() 

In [ ]:
is_spain = temp['Country'] == 'Spain'
temp_spain = temp[is_spain]
temp_spain = temp_spain.groupby(['Country','Date'])['Confirmed'].sum().reset_index()
temp_spain = growth_rate_compute(temp=temp_spain)
fig = go.Figure()
trace1 = go.Scatter(x=temp_spain['Date'],
                    y=temp_spain['Growth_Rate'],
                    mode='lines+markers',
                    hovertext='Spain',
                    marker=dict(color="red",))

fig.add_trace(trace1)
fig.update_layout(title=' Growth Rate Trend in Spain')
fig.show() 

Italy and Spain which are the epicenters of the outbreak in Europe have been witnessing growth rates above 1 consistently. We still are not past the inflection point in these parts of Europe but are pretty close to reaching the peak.

### Growth rate visualization in US and UK

In [ ]:
is_us= temp['Country'] == 'United States'
temp_us = temp[is_us]
temp_us = temp_us.groupby(['Country','Date'])['Confirmed'].sum().reset_index()
temp_us = growth_rate_compute(temp=temp_us)
fig = go.Figure()
trace1 = go.Scatter(x=temp_us['Date'],
                    y=temp_us['Growth_Rate'],
                    mode='lines+markers',
                    hovertext='US',
                    marker=dict(color="red",))

fig.add_trace(trace1)
fig.update_layout(title=' Growth Rate Trend in US')
fig.show() 

In [ ]:
is_uk = temp['Country'] == 'United Kingdom'
temp_uk = temp[is_uk]
temp_uk = temp_uk.groupby(['Country','Date'])['Confirmed'].sum().reset_index()
temp_uk = growth_rate_compute(temp=temp_uk)
fig = go.Figure()
trace1 = go.Scatter(x=temp_uk['Date'],
                    y=temp_uk['Growth_Rate'],
                    mode='lines+markers',
                    hovertext='United Kingdom',
                    marker=dict(color="red",))

fig.add_trace(trace1)
fig.update_layout(title=' Growth Rate Trend in United Kingdom')
fig.show() 

Both US and UK are witnessing growth rates breaching 1 on a daily basis since the first week of March. The trends indicate that both the countries are atleast a few weeks away from their inflection points.

### Growth rate visualization in India

In [ ]:
is_india = temp['Country'] == 'India'
temp_india = temp[is_india]
temp_india = temp_india.groupby(['Country','Date'])['Confirmed'].sum().reset_index()
temp_india = growth_rate_compute(temp=temp_india)
fig = go.Figure()
trace1 = go.Scatter(x=temp_uk['Date'],
                    y=temp_uk['Growth_Rate'],
                    mode='lines+markers',
                    hovertext='India',
                    marker=dict(color="red",))

fig.add_trace(trace1)
fig.update_layout(title=' Growth Rate Trend in India')
fig.show() 

### Growth rate visualization in South Korea

In [ ]:
is_sk = temp['Country'] == 'South Korea'
temp_sk = temp[is_sk]
temp_sk = temp_sk.groupby(['Country','Date'])['Confirmed'].sum().reset_index()
temp_sk = growth_rate_compute(temp=temp_sk)
fig = go.Figure()
trace1 = go.Scatter(x=temp_sk['Date'],
                    y=temp_sk['Growth_Rate'],
                    mode='lines+markers',
                    hovertext='South Korea',
                    marker=dict(color="green",))

fig.add_trace(trace1)
fig.update_layout(title=' Growth Rate Trend in South Korea')
fig.show() 

The containment measures in South Korea seem to be working effectively. The growth rates are stable and contained to less than 1 on most days after March 6th. 

One important thing to note while interpreting the above numbers retrospectively is that the cases observed today are due to infections that have happened roughly 2 to 3 weeks back. Any measures of containment enforced by the government and practised by the people will start bearing fruit after a similar time lag. 

## Closing Commentary


This is certainly not the first pandemic human kind has faced nor will it be the last. 

Be it the 1720 Plague, 1820 Cholera or the 1920 Spanish Flu, each pandemic did take a huge toll on humanity. But after each of these events, human kind found a way to bounce back stronger with renewed vigour. 

One key difference between the current pandemic and the previous ones is that we are armed with much better science and data to understand the impending risk and thereby employ ways to mitigate it.

We definitely are approaching more troubling times over the next couple of months as the data indicates. However countries like China, South Korea and Singapore that were the earliest countries to get affected have shown that with collective effort from both the government and citizens, containment measures can actually succeed. With active government interventions, responsible social behaviour and strict adherence to scientific advice (physical distancing, good hygiene, regular hand washing etc.), we will be able to avoid overwhelming our healthcare systems over the next few months and "flatten the curve" globally.  
